In [3]:
import pandas as pd
from collections import Counter
import spacy,re,json
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy  

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

import re
from pprint import pprint
import os
import warnings
warnings.filterwarnings('ignore')

from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# ============================================================
## 基于 Spacy 的 NER 模型

In [ ]:
PATH ='data/file_ner.json1'

annotated_data = []
lines=[]
with open(PATH, 'r',encoding = 'utf-8 sig') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['text']
    entities = []
    for label in data['labels']:
        entities.append(label)
    annotated_data.append((text, {"entities": data['labels']}))

In [ ]:
training_data=  annotated_data[0:200]
test_data = annotated_data[200:220]

In [ ]:
print(len(training_data), len(test_data))

In [ ]:
training_data[0]

In [ ]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in training_data:
        doc = nlp(text)
#         print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#         print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in training_data:
            doc = nlp2(text)
#             print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#             print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# trained this model with 100 iterations in google colab using GPU runtime with loss: Losses {'ner': 465.24571515528373} 
main(output_dir='ner_model/') 

# ==========================================================
## 导入 NER 模型，识别 JD 中的所有技能类关键词

In [4]:
ner_model = spacy.load('ner_model/')

In [5]:
skill_set = {}
link_set = {}

In [6]:
ind = 0

for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
        
        try:
            #获取文件路径
            my_file = open( os.path.join(root,file),encoding = 'utf-8' )
            my_string = my_file.readlines()

            for string in my_string[1:]:
                doc = ner_model( string )

                temp_list = []
                for chunk in doc.ents:
                    temp_list.append( chunk.text )
                    
                temp_set = set( temp_list )
                
                for tem in list(temp_set):
                    if tem not in skill_set.keys():
                        print( tem )
                        
                    if skill_set.get( tem, 0 ) == 0:
                        skill_set[tem] = 1
                    else:
                        skill_set[tem] += 1
                        
                for t1 in list(temp_set):
                    for t2 in list(temp_set):
                        if t1.text != t2.text:
                            if link_set.get( t1+'|'+t2, 0 ) == 0:
                                link_set[t1+'|'+t2] = 1
                            else:
                                link_set[t1+'|'+t2] += 1
                            

            my_file.close()
        
            if ind % 100 == 0:
                print( '------------------------------------------------------------------------',ind )
                
#             if ind > 2000:
#                 break
                
        except:
            pass

Machine Learning
Python
BlackRock
SQL
R
C++
C
Java
Linux
Data Pre
PHP
Neural Networks
Forecasting
Deep Learning
Minimum 7
Javascript
C#
Hadoop
Spark
TensorFlow
Data Analytics
MapReduce
Monica
MLlib
·
PowerPoint
Statistical
Excel
MATLAB
Athena
as Redshift
S3
SPSS
Tensorflow
SAS
Matlab
Powerpoint
Microsoft Office
NoSQL
non-parametric statistical
Google Analytics
XML
Visualization
AWS
Problem Solving
DMI
Splunk
MA building
Data Modeler
data processing
ETL
Java/Python
Context
Advanced SQL
Exempt
Problem solving
Artificial Intelligence
problem-solving
predictive modeling
Kubernetes
BigQuery
Google Data Studio
Docker
PyTorch
MXNet
Cloud Computing
Tableau
MS Reporting
Analytics
Economics
Elasticsearch
AI
Data Scientist
shell scripting
Oracle
Data Engineering
ECS
KPPs
Data Analysts
JavaScript
deep learning
SPARK
predictive analytics
business intelligence
GCP
NOSQL
wrangle
models
Sequence Modeling
Scala
Microsoft T-SQL
SSIS
NLP
Optimization AI/MI
GGlpot
Redshift
Map Reduce
Nompy
Learning
Data V

internally
Jira
Data Sciences
Wrangling
Master’s
Michigan
Detroit
Caffe
R SSAS
Google Cloud Platform
query performance
AdaBoost
Business analytics
automation
web analytics
refines
Rigorously
Operational Research
Consumer experience
U.Group
HR Analytics
------------------------------------------------------------------------ 2400
Question Answering
Hypothesis testing
DiDi Chuxing
Base
data visualization
architecture
Jave
Digital Industries
SAP SLT
Practical experience building
NH
Manchester
lucid
SQL development
Flask
MS-SQL
Business Process Reengineering
Livonia
On May 7
REST API
BigData and SQL
Spotfire
data analytics
RDS
Presto / Kinesis
DynamoDB
SQS
Kinsesis
customer experience
customer retention
Plano
Kinesis
A full
WE ARE
IL
bioinformaticians
visualization
Mac
Spark / H2O
MS/MA
Salesforce
Database
Google Cloud
DIVERSITY COMMITMENT
troubleshooting
PDA
Modeling
MS Excel
Martindale-Avvo
collaborative filtering
VB
UCSF
Data exploration
Opendoor
----------------------------------------

Syapse
ASR
Predictive modeling
Predictive Modelling
Optimizat
SQL Alchemy
CARE
graphics
Hadoop/HDFS
Python CICD
DevOps (Jenkins
andor predictive
------------------------------------------------------------------------ 7800
sparklyr
finance
machine Learning
Data Prepping
Bonuses
TX 75024
Advanced SAS
DB2
SVN
Java; querying
SERC
The Cognitive
Python/R/SQL
Cloud computing
------------------------------------------------------------------------ 8000
TX 75014
------------------------------------------------------------------------ 8100
Synechron
FinCEN Query
/scripting
TypeScript
Marlboro
NY
OpenNeuro
BIDS
Access Visual Basic
Advanced Google Analytics
database reporting software
database management system software
XMLor SQL
09/09/2019
Data Modelling
------------------------------------------------------------------------ 8300
HP Marketing
CDF
NCR
DNA/RNA/protein
convolutional neural networks
Advanced Python
Provides support
tactical networks
-------------------------------------------------

In [ ]:
# skill_set = dict( Counter(skill_set).most_common(300) )
# skill_set

# ============================================================
## 基于句法分析，识别相关关系

###  若两者为并列关系，或者同位语关系，则输出

In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
def is_father( word, root ):
    if word == root:
        return True
    
    while word != word.head:
        word = word.head
        if word == root:
            return True
    
    return False

In [11]:
ind = 0

total_link_list = []
for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
#         if ind < 500:
#             continue
            
        if ind % 2 == 1:
            continue
        
        my_file = open( os.path.join(root,file),encoding = 'utf-8' )

        my_string = my_file.readlines()
        for string_list in my_string[1:]:
            string_list = string_list.replace('e.g.','like')
            string_list = string_list.replace('.)',')')
#             for string in [sentence for sentence in re.split(r'[？?！!。；;：:\n\r·]', string_list) if sentence]:
            for string in re.split(r'[？?！!。.；;：:\n\r·•*]', string_list):
#                 try:
                doc = nlp( string )

                my_chunks_list = []
                roor_chunks_dict = {}

                for chunk in doc.noun_chunks:
                    if chunk.text in skill_set.keys():
                        my_chunks_list.append( chunk.root )
                        roor_chunks_dict[ str(chunk.root)] = str(chunk)

                if len( my_chunks_list ) > 1:
#                     print( 'skill word:', my_chunks_list )
                    Total_List = []

                    for word in my_chunks_list:
                        temp_list = []
                        temp_list.append(word)

                        pre = word
                        while word != word.head:
                            pre = word
                            relation = word.dep_
                            word = word.head

                            if 'N' in word.tag_ and relation == 'conj':
                                if word in my_chunks_list:
                                    temp_list.append( word )

                                    if len( [str( roor_chunks_dict[str(w)] ) for w in temp_list if 'xperience' not in str(w)] ) == 2:
                                            print(string)
                                            print('result1:') 
                                            print( '-->'.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )
                                            total_link_list.append( [str( roor_chunks_dict[str(w)] ) for w in temp_list] )
                                            print()

                                break
                
        my_file.close()

        if ind % 500 == 0:
            print( '==================================================',ind )
            
        if ind > 500:
            break

 The candidate should have a minimum of 3 years of solid work experience in a professional organization/corporation using Python, SQL, Python libraries, Machine Learning Algorithms ( i
result1:
SQL-->Python

The successful candidate will have a passion for technology, comfort working across a range of technology capabilities, including databases, software, and infrastructure, and a love of solving complex problems
result1:
software-->databases

Bachelors degree required, Masters degree preferred, in a quantitative discipline (computer science, mathematics, statistics, economics, physics, engineering or related discipline)
result1:
economics-->statistics

The opportunity is to join our New York team as an analyst, associate, or VP, with a focus on applications of machine learning and artificial intelligence in fixed income/securitized products markets
result1:
artificial intelligence-->machine learning

Responsible for applying machine learning/artificial intelligence and big data techn

Python, Julia, SQL, Hadoop, Hive, and Apache Spark
result1:
Julia-->Python

Python, Julia, SQL, Hadoop, Hive, and Apache Spark
result1:
SQL-->Julia

Python, Julia, SQL, Hadoop, Hive, and Apache Spark
result1:
Hadoop-->SQL

Python, Julia, SQL, Hadoop, Hive, and Apache Spark
result1:
Hive-->Hadoop

Python, Julia, SQL, Hadoop, Hive, and Apache Spark
result1:
Apache Spark-->Hive

Experience building data models, and managing ML pipelines with tools such as Python, Julia, and Apache Spark
result1:
Julia-->Python

Experience building data models, and managing ML pipelines with tools such as Python, Julia, and Apache Spark
result1:
Apache Spark-->Julia

      3+ years work experience in a big data domain including, data visualization, machine learning, data mining, preparation and modeling in both regression and classification
result1:
machine learning-->data visualization

       Expert ability with Python and R data science libraries to include at least Pandas, SciKit Learn, Keras, and MLli

 Senior Data Engineer ( data scientist ) 6 months + contract O'hare Airport - Rosemont Face to face interview REQUIRED QUALIFICATIONS BS in Mathematics, Computer Science, Information Management, Statistics or a Related Technical Field 3+ years’ experience with metrics and scorecard development and deployment 3+ years’ experience in the development of business metrics 2+ years’ experience in healthcare industry Strong understanding of SQL and able to simplify complex data sets Experience with Amazon Redshift Hands on experience with R and/or Python (data extraction, manipulation, statistical analysis and predictive modeling) Ability to understand business strategy, provide consultative business analysis, and leverage technical skills to create insightful, effective BI solutions Ability to thrive in a project-based environment with evolving requirements Strong analytical skills with the ability to collect, aggregate, analyze, and disseminate significant amounts of information with attent

  Master’s degree or higher in data science, math, engineering, or a related field is requied
result1:
engineering-->math

 Working knowledge of tools such as R or MatLab is a plus
result1:
MatLab-->R

 Masters degree from an accredited college/university in Data Science, Machine Learning, Computer Science, Computational Linguistics, Statistics, Mathematics, Engineering, Bioinformatics, Physics, Operations Research or related fields, plus at least two years of work experience in a commercial setting using data science skills
result1:
Mathematics-->Statistics

 Masters degree from an accredited college/university in Data Science, Machine Learning, Computer Science, Computational Linguistics, Statistics, Mathematics, Engineering, Bioinformatics, Physics, Operations Research or related fields, plus at least two years of work experience in a commercial setting using data science skills
result1:
Engineering-->Mathematics

 Should be able to code in python or R to replicate sensitivity analy

 BS and Masters degrees in computer science, or related technical, math, or scientific field 
result1:
math-->computer science

 Strong working knowledge of deep learning, machine learning and statistics
result1:
machine learning-->machine learning

 Strong working knowledge of deep learning, machine learning and statistics
result1:
statistics-->machine learning

 Hands on experience building models with deep learning frameworks like MXNet, Tensorflow, Caffe, Torch, Theano or similar
result1:
Tensorflow-->MXNet

 Hands on experience building models with deep learning frameworks like MXNet, Tensorflow, Caffe, Torch, Theano or similar
result1:
Caffe-->Tensorflow

 Hands on experience building models with deep learning frameworks like MXNet, Tensorflow, Caffe, Torch, Theano or similar
result1:
Torch-->Caffe

 Hands on experience building models with deep learning frameworks like MXNet, Tensorflow, Caffe, Torch, Theano or similar
result1:
Theano-->Torch

 Experience in using Python, R or M

...

 access to online and onsite training in data analysis and presentation methodologies, and tools like Hortonworks, Docker, Tableau, and Splunk a chance to change the world with the Data Science Bowl—the world’s premier data science for social good competition participation in partnerships with data science leaders, like our partnership with NVIDIA to deliver Deep Learning Institute (DLI) training to the federal government You’ll have access to a wealth of training resources through our Analytics University, an online learning portal specifically geared towards data science and analytics skills, where you can access more than 5000 functional and technical courses, certifications, and books
result1:
Tableau-->Docker

  Technical Bachelor’s or Master’s Degree preferably in Computer Science, Information Systems, Statistical Analysis or Economics Familiarity with statistical analysis/modeling, data science and/or data design and development of actionable advanced analytical solutions that p

JOB SUMMARY Responsible for scoping, developing, modeling, implementing, and delivering robust data analytics by using Teradata Aster MAJOR DUTIES AND RESPONSIBILITIES Actively and consistently support all efforts to simplify and enhance the customer experience Understand customer business use cases and be able to translate them to technical specifications and vision on how to implement a solution Clearly communicate the benefits of the analytic solutions to both a business as well as a technical audience Leverage knowledge in analytic and statistical algorithms to help customers explore methods to improve their business Lead large-scale analytical research projects through all stages including concept formulation, determination of appropriate statistical methodology, data manipulation, research evaluation, and final research report Design, build, and extract large and complex data sets while thinking strategically about uses of data and how data use interacts with data design Lead lar

 Required Technical and Professional Expertise BA/BS or MS in Data Science, Machine Learning, Statistics, or related STEM field Proficiency in Python, R, SQL or another programming language Experience or coursework in machine learning Statistical modeling through software (like SPSS) or programming language (like Python) Understanding of database and analytical technologies in the industry Demonstrated ability to think strategically about business, product and technical challenges in an enterprise environment Excellent oral and written communication skills Ability to collaborate in a team environment Proven ability to collaborate and develop positive business relationships both internally and externally Familiarity with Jira, etc
result1:
Machine Learning-->Data Science

 Required Technical and Professional Expertise BA/BS or MS in Data Science, Machine Learning, Statistics, or related STEM field Proficiency in Python, R, SQL or another programming language Experience or coursework in 

result1:
SQL-->Business Analysis

 Senior Data Scientist, Statistical Analysis, Business Analysis, SQL, R, SAS, Database Design, Data Modeling, Python, SQL, Process Improvement, Reporting Job ID
result1:
R-->SQL

 Senior Data Scientist, Statistical Analysis, Business Analysis, SQL, R, SAS, Database Design, Data Modeling, Python, SQL, Process Improvement, Reporting Job ID
result1:
SAS-->R

 Senior Data Scientist, Statistical Analysis, Business Analysis, SQL, R, SAS, Database Design, Data Modeling, Python, SQL, Process Improvement, Reporting Job ID
result1:
Database Design-->SAS

 Senior Data Scientist, Statistical Analysis, Business Analysis, SQL, R, SAS, Database Design, Data Modeling, Python, SQL, Process Improvement, Reporting Job ID
result1:
Data Modeling-->Database Design

 Senior Data Scientist, Statistical Analysis, Business Analysis, SQL, R, SAS, Database Design, Data Modeling, Python, SQL, Process Improvement, Reporting Job ID
result1:
Python-->Data Modeling

 Senior Data Scien

  3+ years of work experience in biotechnology role (like as a statistician, statistical programmer, data analyst, data scientist, computational biologist, bioinformatician) Experience with programming (like R, MATLAB, Python) and database languages (like SQL), as well as data visualization tools (like Tableau, Spotfire) Experience with statistical data analysis such as linear/non-linear multivariate models and hypothesis testing Applied experience with implementing machine learning solutions Experience articulating business questions and using mathematical and programmatic solutions to arrive at an answer Demonstrated skills in selecting the right tools given a data analysis question Demonstrated effective written and verbal communication skills Demonstrated leadership and self-direction
result1:
MATLAB-->R

  3+ years of work experience in biotechnology role (like as a statistician, statistical programmer, data analyst, data scientist, computational biologist, bioinformatician) Exper

In [13]:
ind = 0

total_link_list = []
for root,dirs,files in os.walk(r"E:\dataset\dataset"):
    for file in files:
        ind += 1
#         if ind < 500:
#             continue
            
        if ind % 2 == 1:
            continue
        
        my_file = open( os.path.join(root,file),encoding = 'utf-8' )

        my_string = my_file.readlines()
        for string_list in my_string[1:]:
            string_list = string_list.replace('e.g.','like')
            string_list = string_list.replace('.)',')')
#             for string in [sentence for sentence in re.split(r'[？?！!。；;：:\n\r·]', string_list) if sentence]:
            for string in re.split(r'[？?！!。.；;：:\n\r·•*]', string_list):
#                 try:
                doc = nlp( string )

                my_chunks_list = []
                roor_chunks_dict = {}

                for chunk in doc.noun_chunks:
                    if chunk.text in skill_set.keys():
                        my_chunks_list.append( chunk.root )
                        roor_chunks_dict[ str(chunk.root)] = str(chunk)

                if len( my_chunks_list ) > 1:
#                     print( 'skill word:', my_chunks_list )
                    Total_List = []

                    for word in my_chunks_list:
                        temp_list = []
                        temp_list.append(word)

                        pre = word
                        if word != word.head:
                            pre = word
                            relation = word.dep_
                            word = word.head

                            if 'N' in word.tag_ and relation == 'conj':
                                if word in my_chunks_list:
                                    temp_list.append( word )

                                    if len( [str( roor_chunks_dict[str(w)] ) for w in temp_list if 'xperience' not in str(w)] ) == 2:
#                                             print(string)
                                            print('result1:') 
                                            print( '-->'.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )
                                            total_link_list.append( [str( roor_chunks_dict[str(w)] ) for w in temp_list] )
                                            print()

                                break
                
        my_file.close()

        if ind % 500 == 0:
            print( '==================================================',ind )

result1:
SQL-->Python

result1:
software-->databases

result1:
artificial intelligence-->machine learning

result1:
artificial intelligence-->machine learning

result1:
artificial intelligence-->machine learning

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
Spark-->Python

result1:
Spark-->Python

result1:
Deep Learning-->AI

result1:
Python-->PHP

result1:
Spark-->Python

result1:
Mathematics-->Statistics

result1:
data-->research

result1:
R-->python

result1:
R-->python

result1:
analytics-->data

result1:
R-->Python

result1:
Julia-->Python

result1:
Julia-->Python

result1:
machine learning-->data visualization

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
data modeling-->analysis

result1:
C++-->C

result1:
Machine Learning-->Artificial Intelligence

result1:
Scala-->Spark

result1:
Mathematics-->Statistics

result1:
Statistics-->Economics

result1:
Sales-->Marketing

result1:
SPSS-->Excel

result1:
Python-->R

result1:
Docker-->GitHub

res

================================================== 500
result1:
R-->Python

result1:
analysis-->statistical modeling

result1:
R-->Python

result1:
Matlab-->Python

result1:
Theano-->TensorFlow

result1:
Spark-->Hadoop

result1:
machine learning-->text analytics

result1:
Engineering-->Statistics

result1:
R-->Python

result1:
data visualization-->Qlik

result1:
Mathematics-->Statistics

result1:
R-->SAS

result1:
deep learning-->deep learning

result1:
AI-->Machine Learning

result1:
statistical analysis-->predictive modeling

result1:
PowerPoint-->Outlook

result1:
Python-->R

result1:
R-->machine learning

result1:
machine learning-->statistics

result1:
Analytics-->Data Science

result1:
R-->SAS

result1:
Python-->R

result1:
Spark-->Hadoop

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
Sales-->Product Management

result1:
R-->Python

result1:
NLP-->data analysis

result1:
NLP-->data analysis

result1:
software-->algorithms

result1:
statistical analysis-->statistical an

result1:
Python-->Java

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
Mathematics-->Statistics

result1:
R-->Python

result1:
Apache Spark-->SQL

result1:
Engineering-->Data Science

result1:
R-->Python

result1:
Hadoop-->SQL

result1:
dashboards-->business intelligence

result1:
Data Analysis-->Machine Learning

result1:
Analytics-->Data Science

result1:
Spark-->Hadoop

result1:
Python-->Java

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
machine learning-->data analytics

result1:
R-->Python

result1:
Economics-->Finance

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
R-->python

result1:
Excel-->Access

result1:
Python-->SQL

result1:
R-->Python

result1:
R-->Python

result1:
Operations Research-->Mathematics

result1:
R-->Python

result1:
Hadoop-->HDFS

result1:
Python-->Java

result1:
Python-->R

result1:
Economics-->Statistics

result1:
SAS-->R

result1:
R-->Python

result1:
statistics-->analytics

result1:
SQL-->SAS

result1:
SPSS-->SAS

result1:

result1:
Python-->R

result1:
AWS-->Hadoop

result1:
statistical algorithms-->machine learning

result1:
Software Development-->Engineering

result1:
Visualization-->Analytics

result1:
R-->SAS

result1:
Statistics-->Mathematics

result1:
Python-->R

result1:
R-->Apache Spark

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
Spark-->S3

result1:
Spark-->S3

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
Spark-->S3

result1:
Spark-->S3

result1:
Python-->R

result1:
Mathematics-->Statistics

result1:
statistics-->machine learning

result1:
Python-->R

result1:
Python-->Java

result1:
svn-->git

result1:
Operations Research-->Mathematics

result1:
SQL-->Microsoft Excel

result1:
marketing-->sales

result1:
deep learning-->deep learning

result1:
deep learning-->deep learning

result1:
Redshift-->SQLite

result1:
Word-->Outlook

result1:
Hadoop-->Big Data

result1:
artificial intelligence-->machine l

result1:
Statistics-->Machine Learning

result1:
Python-->Java

result1:
Hadoop-->Spark

result1:
statistics-->machine learning

result1:
math-->CS

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
data-->Kafka

result1:
Spark-->Hadoop

result1:
Hadoop-->SQL

result1:
machine learning-->statistical modeling

result1:
machine learning-->statistical modeling

result1:
Economics-->Data analytics

result1:
Python-->R

result1:
Economics-->Data analytics

result1:
Python-->R

result1:
statistics-->computer science

result1:
deep learning-->deep learning

result1:
R-->Python

result1:
statistics-->computer science

result1:
statistics-->computer science

result1:
deep learning-->deep learning

result1:
R-->Python

result1:
statistics-->computer science

result1:
dashboards-->analytics

result1:
hypothesis testing-->machine learning

result1:
Spark-->S3

result1:
statistics-->algorithms

result1:
Python-->R

result1:
Business-->Statistical Analysis

result1:
algorithms-->models

resul

...

result1:
machine learning-->statistics

result1:
Operations Research-->Data Science

result1:
SQL-->Oracle

result1:
Operations Research-->Data Science

result1:
Python-->R

result1:
Python-->R

result1:
Mathematics-->Analytics

result1:
Python-->R

result1:
Hadoop-->Teradata

result1:
Word-->Excel

result1:
design-->data analysis

result1:
Python-->MATLAB

result1:
deep learning-->deep learning

================================================== 8000
result1:
Keras-->TensorFlow

result1:
models-->data

result1:
SVN-->Git

result1:
predictive analysis-->statistical algorithms

result1:
Mathematics-->computer science

result1:
data analysis-->data analysis

result1:
Mathematics-->computer science

result1:
data analysis-->data analysis

result1:
JAVA-->Javascript

result1:
S3-->Redshift

result1:
Kafka-->Spark

result1:
AI-->Machine learning

result1:
R-->Python

result1:
SQL Server-->Oracle

result1:
machine learning-->statistics

result1:
Econometrics-->Statistics

result1:
Statistics

result1:
Theano-->TensorFlow

result1:
Mathematics-->Statistics

result1:
finance-->marketing

result1:
Engineering-->Statistics

result1:
Artificial Intelligence-->Machine Learning

result1:
statistics-->Advanced degree

result1:
Python-->R

result1:
Python-->R

result1:
Python-->R

result1:
Machine Learning-->data

result1:
Matlab-->Python

result1:
machine learning-->advanced analytics

result1:
statistics-->computer science

result1:
statistical analysis-->machine learning

result1:
Jupyter-->Python

result1:
Python-->SAS

result1:
Data Science-->Data Analytics

result1:
Statistics-->Mathematics

result1:
automation-->analytics

result1:
Predictive Modeling-->Machine Learning

result1:
R-->Python

result1:
Statistics-->Machine Learning

result1:
SQL-->Python

result1:
predictive modeling-->machine learning

result1:
Marketing-->Product Management

result1:
machine learning-->advanced analytics

result1:
statistics-->computer science

result1:
statistical analysis-->machine learning

result1:
deep learning-->deep learning

result1:
R-->Python

result1:
Python-->Java

result1:
Python-->R

result1:
Azure-->AWS

result1:
Python-->Java

result1:
SAS-->R

result1:
SAS-->R

result1:
Python-->Java

result1:
simulations-->algorithms

result1:
machine learning-->predictive modeling

result1:
AWS-->Azure

result1:
data modeling-->databases

result1:
simulations-->algorithms

result1:
machine learning-->predictive modeling

result1:
AWS-->Azure

result1:
data modeling-->databases

result1:
deep learning-->NLP

result1:
Scala-->Python

result1:
JavaScript-->Python

result1:
Azure-->AWS

result1:
visualization-->analysis

result1:
Hive-->Hadoop

result1:
Python-->Spark

result1:
machine-->statistics

result1:
machine-->statistics

result1:
Economics-->Mathematics

result1:
Java-->Python

result1:
SAS-->R

result1:
R-->Python

result1:
machine learning-->statistical modeling

result1:
SQL-->relational databases

result1:
Predictive Analytics-->Data Science

result1:
collaborativ

result1:
Scala-->Java

result1:
machine-->Big Data

result1:
Python-->R

result1:
H2O-->R

result1:
MapReduce-->Hadoop

result1:
SQL-->Python

result1:
Scala-->Python

result1:
machine-->Big Data

result1:
Python-->R

result1:
Python-->Java

result1:
Excel-->Access

result1:
statistical modeling-->machine learning

result1:
Java-->python

result1:
Dask-->Spark

result1:
Hive-->Pig

result1:
Java-->Python

result1:
Hive-->Pig

result1:
Java-->Python

result1:
Economics-->Statistics

result1:
Spark-->python

result1:
Mahout-->Hadoop

result1:
Data Science-->Statistics

result1:
Hadoop-->SQL

result1:
Predictive Modeling-->Machine Learning

result1:
Scala-->Python

result1:
Spark-->Hadoop

result1:
Excel-->Word

result1:
Tensorflow-->PyTorch

result1:
Ruby-->Python

result1:
deep learning-->deep learning

result1:
Python-->R

result1:
Statistics-->Mathematics

result1:
Marketing-->Credit

result1:
R-->SAS

result1:
SQL-->SAS

result1:
statistics-->economics

result1:
Engineering-->Statist

result1:
Python-->R

result1:
AI-->machine learning

result1:
Pandas-->Scikit-Learn

result1:
SQL-->Python

result1:
Caffe-->TensorFlow

result1:
SQL-->Python

result1:
Scala-->Python

result1:
Spark-->Hadoop

result1:
Python-->R

result1:
Python-->Java

result1:
C++-->Java

result1:
Math-->Engineering

result1:
OpenShift-->Docker

result1:
Python-->Java

result1:
statistics-->machine learning

result1:
Scala-->Python

result1:
machine learning-->statistics

result1:
Ph-->MS

result1:
Mathematics-->Statistics

result1:
R-->SQL

result1:
analysis-->predictive modeling

result1:
Mathematics-->Statistics

result1:
data analytics-->business intelligence

result1:
data analytics-->computer science

result1:
SPSS-->Power BI

result1:
AI-->machine learning

result1:
R-->Python

result1:
Power Point-->Excel

result1:
Jupyter Notebooks-->Spark

result1:
Economics-->Mathematics

result1:
Advanced Analytics-->Data

result1:
SAS-->R

result1:
artificial intelligence-->machine learning

result1:
St

In [15]:
link_df = pd.DataFrame( total_link_list )
link_df = link_df.rename(columns = {0:'node1',1:'node2'})
link_df = link_df.drop_duplicates()
link_df

,node1,node2
0,SQL,Python
1,software,databases
2,artificial intelligence,machine learning
5,Python,R
6,Mathematics,Statistics
...,...,...
8192,Python,BigQuery
8202,Azure,Python
8213,Django,Python
8216,TensorFlow,R


In [16]:
link_df.to_excel(r'E:\dataset\Correlation_Link_.xls',encoding = 'utf-8 sig',index = False)

# 关系绘制

In [17]:
from unittest.mock import patch
from nose.tools import assert_equal, assert_in
from pyecharts import options as opts
from pyecharts.charts.basic_charts.graph import Graph

from pyecharts import options as opts
from pyecharts.charts import Graph

nodes = []
links = []

nodes_set = []

for ind, row in link_df.iterrows():
    
    if row['node1'] != row['node2']:
        
        if row['node1'] not in nodes_set:
            nodes_set.append( row['node1'] )
            nodes.append( {'name': row['node1'] , "symbolSize": 15 + skill_set[row['node1']] / 20 } )
            
        if row['node2'] not in nodes_set:
            nodes_set.append( row['node2'] )
            nodes.append( {'name': row['node2'] , "symbolSize": 15 + skill_set[row['node2']] / 20 } )
        
        links.append({"source": row['node2'], "target": row['node1']})

Graph().add(
    "", 
    nodes, 
    links, 
    repulsion=8000,
#     linestyle_opts=opts.LineStyleOpts(width=2, curve=0.1, opacity=0.9),
    linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
    label_opts=opts.LabelOpts( ),
    categories=None,
    is_focusnode=True,
#     layout="none",
    is_roam=True
    
).set_global_opts(title_opts=opts.TitleOpts(title="")).render_notebook()